In [29]:
import pandas as pd
import random
import math

# read in time point 1 for training
# train on it - model 1
# test on tp 2
# read tp 2
# add it to xgb - model 2
# test on tp 3
# read tp 3
# add it to xgb - model 3
# test on tp 4
# test model 1
data = pd.read_csv('NY_trials_2002-2022_conv_agg_updated.csv')
data.head()

,State,City,Date Sown,Brand,Variety,Date of Cut,Julian Day,Plant Lodge,Plant Height,Yield % or Vernal,...,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55
0,NY,Ithaca,5/10/02,NaN,Seedway 9558,6/15/05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NY,Ithaca,5/10/02,NaN,HYTEST 410,6/15/05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NY,Ithaca,5/10/02,NaN,Paragon BR,6/15/05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NY,Ithaca,5/10/02,NaN,4A421,6/15/05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NY,Ithaca,5/10/02,NaN,WL 319HQ,6/15/05,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def create_class(df):
    # Create a DataFrame
    print("Length of Dataset before class column: ", len(df))
    # Calculate the mean and standard deviation for the Yield column
    mean_yield = df['Yield (tons/acre)'].mean()
    std_yield = df['Yield (tons/acre)'].std()

    # Define the thresholds based on standard deviations
    lower_threshold = mean_yield - std_yield  # Yield less than -1 stdv below mean
    upper_threshold = mean_yield + std_yield  # Yield more than +1 stdv above mean

    # Classify yields based on the thresholds
    def classify_yield(yield_value):
        if yield_value < lower_threshold:
            return 0  # Low Yield (Class 1)
        elif lower_threshold <= yield_value <= upper_threshold:
            return 1  # Medium Yield (Class 2)
        else:
            return 2  # High Yield (Class 3)

    # Apply the classification function to the Yield column
    df['class'] = df['Yield (tons/acre)'].apply(classify_yield)

    # Display the result
    #print(df)
    print("Length of Dataset after: ", len(df))
    return df



In [5]:
def rename_columns(df):
    
    columns_to_keep=['Yield_Diff', 'Total Solar Radiation (W/m^2)', 'Total precipitation (mm)',
       'Avg Min Temp (C)', 'Avg Max Temp (C)', 'City', 'class', 'Date of Cut']

    df=df[columns_to_keep]
    #print(df.head())
    # Rename the column
    df.rename(columns={'Total Solar Radiation (W/m^2)': 'radiation'}, inplace=True)
    df.rename(columns={'Total precipitation (mm)': 'rain'}, inplace=True)
    #df = df.rename(columns={'Yield_Diff':'yield'})
    #data = data.rename(columns={'Total Radiation (W/m^2)':'radiation'})
    #data = data.rename(columns={'Total Rainfall (mm)':'rain'})
    df = df.rename(columns={'Yield_Diff': 'yield'})
    df = df.rename(columns={'Avg Min Temp (C)':'avg_min_temp'})
    df = df.rename(columns={'Avg Max Temp (C)':'avg_max_temp'})
    df = df.rename(columns={'Date of Cut':'Year'})
    return df

In [8]:
def data_cleaning(df):
    columns_to_keep=['Yield (tons/acre)', 'Total Solar Radiation (W/m^2)', 'Total precipitation (mm)',
       'Avg Min Temp (C)', 'Avg Max Temp (C)', 'City', 'Date of Cut']

    df=df[columns_to_keep]
    #print(df.head())
    # Rename the column
    df.rename(columns={'Total Solar Radiation (W/m^2)': 'radiation'}, inplace=True)
    df.rename(columns={'Total precipitation (mm)': 'rain'}, inplace=True)
    #df3 = df3.rename(columns={'Yield_Diff':'yield'})
    #data = data.rename(columns={'Total Radiation (W/m^2)':'radiation'})
    #data = data.rename(columns={'Total Rainfall (mm)':'rain'})
    df = df.rename(columns={'Avg Min Temp (C)':'avg_min_temp'})
    df = df.rename(columns={'Avg Max Temp (C)':'avg_max_temp'})
    df = df.rename(columns={'Date of Cut':'Year'})
    return df


In [9]:
def split_dataset_for_City(city,df):
    city_datasets = {city: city_df for city, city_df in df.groupby("City")}
    data=city_datasets.get(city)
    return data

In [10]:
# Group by 'Year' and take the mean for numeric columns
def create_consise_dataset(df):
    df_grouped = df.groupby("Year", as_index=False).mean(numeric_only=True)

    # Add back the 'City' column (if it's the same for all rows in a group)
    df_grouped["City"] = df.groupby("Year")["City"].first().values

    return df_grouped




In [11]:
def convert_date_to_year(df):
    df['Year'] = pd.to_datetime(df['Year'], errors='coerce')
    # Now extract the year
    df['Year'] = df['Year'].dt.year
    return df

In [12]:
pip install imbalanced-learn



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
from imblearn.over_sampling import SMOTE
import pandas as pd

from imblearn.over_sampling import SMOTE
import pandas as pd

def apply_smote(df, target_column):
    # Separate features (X) and target (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Count occurrences of each class
    class_counts = y.value_counts()

    # Filter out classes with only 1 sample (since SMOTE cannot handle them)
    valid_classes = class_counts[class_counts > 1].index
    df_filtered = df[df[target_column].isin(valid_classes)]

    # Update X, y after filtering
    X_filtered = df_filtered.drop(columns=[target_column])
    y_filtered = df_filtered[target_column]

    # Set dynamic k_neighbors (ensure it's valid)
    min_class_size = class_counts.min()
    k_neighbors = max(1, min(min_class_size - 1, 5))  # Ensures valid k_neighbors

    # Apply SMOTE only if at least 2 classes remain
    if len(y_filtered.unique()) > 1:
        smote = SMOTE(sampling_strategy="auto", random_state=42, k_neighbors=k_neighbors)
        X_resampled, y_resampled = smote.fit_resample(X_filtered, y_filtered)

        # Convert back to DataFrame
        df_resampled = pd.DataFrame(X_resampled, columns=X_filtered.columns)
        df_resampled[target_column] = y_resampled
    else:
        print("SMOTE not applied: Not enough class diversity after filtering.")
        return df_filtered

    return df_resampled



In [14]:
import pandas as pd

def balance_data_by_year_upsample(df):
    """Ensures equal number of data points for each year by upsampling to the maximum year count."""
    
    # Check if 'Year' exists
    if "Year" not in df.columns:
        raise KeyError("The dataset does not contain a 'Year' column.")
    
    # Group by 'Year' and get the count of records per year
    year_counts = df['Year'].value_counts()
    #print(year_counts)
    # Get the maximum number of records across all years
    max_count = year_counts.max()
    #print(max_count)
    # Initialize an empty list to store balanced data
    balanced_data = []

    # Iterate over each year and sample the same number of records
    for year in year_counts.index:
        year_data = df[df['Year'] == year]
        
        # If there are fewer data points than the maximum, upsample to the max_count
        if len(year_data) < max_count:
            year_data = year_data.sample(n=max_count, replace=True, random_state=42)
            #print("Upsampling")
        # Append the sampled data to the list
        balanced_data.append(year_data)
    #print(balanced_data)
    # Concatenate all the sampled data into a single DataFrame
    balanced_df = pd.concat(balanced_data, axis=0)
    
    return balanced_df




In [30]:
data=data_cleaning(data)
#print(data.head())
data=split_dataset_for_City('Cobleskill',data)
data=create_consise_dataset(data)
# Sort by 'Year' to maintain the chronological order
data=convert_date_to_year(data)
data = data.sort_values(by="Year")
#print("Before upsampling")
#print("Before undersampling")
print("Before SMOTE")
data.drop(columns=['City'], inplace=True)
print(data.head())

print(data["Year"].value_counts())  # Check class balance after SMOTE

#data=balance_data_by_year_upsample(data)
#data=undersampling(data)
#print("After Undersampling")
data=apply_smote(data,'Year')



print("After SMOTE")
print(data["Year"].value_counts())  # Check class balance after SMOTE
data=create_class(data)
data = data.sort_values(by="Year")
data.reset_index(drop=True, inplace=True)
print(len(data))
print(data.head())
#print(data.head())

Before SMOTE
    Year  Yield (tons/acre)  radiation     rain  avg_min_temp  avg_max_temp
3   2005           1.171667  348407.31  3980.27        1.5414       13.0870
28  2005           1.390000  373548.92  4196.37        2.2246       13.8196
9   2005           1.309167  361326.06  4070.12        1.8320       13.4049
45  2005           0.736667  389742.63  4309.98        2.6606       14.3005
27  2006           1.854800   30625.83   491.76       11.5036       22.9716
Year
2005    4
2007    4
2008    4
2010    4
2018    4
2009    3
2011    3
2014    3
2015    3
2017    3
2019    3
2020    3
2021    3
2006    2
2016    1
Name: count, dtype: int64
After SMOTE
Year
2005    4
2006    4
2007    4
2008    4
2009    4
2010    4
2011    4
2014    4
2015    4
2017    4
2018    4
2019    4
2020    4
2021    4
Name: count, dtype: int64
Length of Dataset before class column:  56
Length of Dataset after:  56
56
   Yield (tons/acre)      radiation         rain  avg_min_temp  avg_max_temp  \
0           

/var/folders/zn/_j79f2r529s6w4hp6tkpln8c0000gn/T/ipykernel_29149/1748844638.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Total Solar Radiation (W/m^2)': 'radiation'}, inplace=True)
/var/folders/zn/_j79f2r529s6w4hp6tkpln8c0000gn/T/ipykernel_29149/1748844638.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Total precipitation (mm)': 'rain'}, inplace=True)
/var/folders/zn/_j79f2r529s6w4hp6tkpln8c0000gn/T/ipykernel_29149/3914005513.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consist

In [31]:
print(data['Year'].unique())

[2005 2006 2007 2008 2009 2010 2011 2014 2015 2017 2018 2019 2020 2021]


In [32]:
data_Df=data[data['Year'] != 2021]
data_Df = data_Df.rename(columns={'Yield (tons/acre)':'yield'})
#print(len(target_yearDf))
#print(target_yearDf.head())
# Display the filtered DataFrame
data_Df=data_Df.drop(columns={'Year'}, inplace=False)
#print(final_yearDf)
print(data_Df.head())

      yield      radiation         rain  avg_min_temp  avg_max_temp  class
0  1.171667  348407.310000  3980.270000      1.541400     13.087000      0
1  1.390000  373548.920000  4196.370000      2.224600     13.819600      0
2  1.309167  361326.060000  4070.120000      1.832000     13.404900      0
3  0.736667  389742.630000  4309.980000      2.660600     14.300500      0
4  1.636643   36288.621126   559.345768     11.542541     23.031124      1


In [33]:
target_Df=data[data['Year'] == 2021]
target_Df = target_Df.rename(columns={'Yield (tons/acre)':'yield'})
#print(len(target_yearDf))
#print(target_yearDf.head())
# Display the filtered DataFrame
target_Df=target_Df.drop(columns={'Year'}, inplace=False)
#print(final_yearDf)
print(target_Df.head())

       yield      radiation         rain  avg_min_temp  avg_max_temp  class
52  1.778000  387164.580000  3839.540000      3.349100     14.571000      1
53  2.491000  355017.680000  3437.960000      2.503000     13.708100      2
54  1.482000  367072.990000  3602.460000      2.825400     14.053800      1
55  2.285712  357470.417365  3471.428679      2.568595     13.778435      1


In [34]:
print(len(target_Df))

4


In [35]:
all_yearsDf=data
all_yearsDf = all_yearsDf.rename(columns={'Yield (tons/acre)':'yield'})
#print(len(target_yearDf))
#print(target_yearDf.head())
# Display the filtered DataFrame
all_yearsDf=all_yearsDf.drop(columns={'Year'}, inplace=False)
#print(final_yearDf)
print(all_yearsDf.head())

      yield      radiation         rain  avg_min_temp  avg_max_temp  class
0  1.171667  348407.310000  3980.270000      1.541400     13.087000      0
1  1.390000  373548.920000  4196.370000      2.224600     13.819600      0
2  1.309167  361326.060000  4070.120000      1.832000     13.404900      0
3  0.736667  389742.630000  4309.980000      2.660600     14.300500      0
4  1.636643   36288.621126   559.345768     11.542541     23.031124      1


In [36]:
print(len(all_yearsDf))

56


In [37]:
target_Df=target_Df.dropna()
all_yearsDf=all_yearsDf.dropna()
data_Df=data_Df.dropna()


In [38]:
target_Df.reset_index(drop=True, inplace=True)
all_yearsDf.reset_index(drop=True, inplace=True)
data_Df.reset_index(drop=True, inplace=True)

In [39]:
print(target_Df.head())
print(len(target_Df))

      yield      radiation         rain  avg_min_temp  avg_max_temp  class
0  1.778000  387164.580000  3839.540000      3.349100     14.571000      1
1  2.491000  355017.680000  3437.960000      2.503000     13.708100      2
2  1.482000  367072.990000  3602.460000      2.825400     14.053800      1
3  2.285712  357470.417365  3471.428679      2.568595     13.778435      1
4


In [40]:
print(all_yearsDf.head())
print(len(all_yearsDf))

      yield      radiation         rain  avg_min_temp  avg_max_temp  class
0  1.171667  348407.310000  3980.270000      1.541400     13.087000      0
1  1.390000  373548.920000  4196.370000      2.224600     13.819600      0
2  1.309167  361326.060000  4070.120000      1.832000     13.404900      0
3  0.736667  389742.630000  4309.980000      2.660600     14.300500      0
4  1.636643   36288.621126   559.345768     11.542541     23.031124      1
56


In [41]:
data_Df.to_csv("data_new.csv", index=False)

In [42]:
print(len(data_Df))

52


In [43]:
data1=data_Df

In [44]:
# from sdv.tabular import TVAE

# model = TVAE()
# model.fit(data)
import numpy as np
samples_out = 20000 # total number of samples/records to generate/synthesize
no_stds = 1 # number of standard deviations within which synthesized values must fall
number_of_classes = (data1['class'].unique()).size # number of unique classes in input data

data_len = len(data1.index)
F = [] # a list of the feature vectors dataframes, one per class
for class_no in range(number_of_classes):
    df = pd.DataFrame(data1[data1['class'] == class_no])
    #print("In F: ")
    #print(check_nan_in_dataframe(df))
    F.append(df)


def synthesize_tabular_data(F, samples_out, no_stds, no_classes, no_records):
    new_F = []
    for index, entry in enumerate(F):
        # Clean NaN values before processing
        entry = entry.dropna(subset=['yield', 'radiation', 'rain', 'avg_max_temp', 'avg_min_temp'])

        # Check if there's data left after dropping NaN values
        if len(entry) == 0:
            print(f"Warning: Class {index} has no valid data after removing NaNs.")
            continue
        yield_ = entry['yield']
        mean_yield = yield_.mean()
        std_yield = yield_.std()
        total_rad = entry['radiation']
        mean_rad = total_rad.mean()
        std_rad = total_rad.std()
        total_rain = entry['rain']
        mean_rain = total_rain.mean()
        std_rain = total_rain.std()
        avg_max_temp = entry['avg_max_temp']
        mean_max_temp = avg_max_temp.mean()
        std_max_temp = avg_max_temp.std()
        avg_min_temp = entry['avg_min_temp']
        mean_min_temp = avg_min_temp.mean()
        std_min_temp = avg_min_temp.std()
        
        #print(str(mean_yield) + " " + str(std_yield) + " " + str(mean_rad) + " " + str(std_rad) + " " + str(mean_rain) + " " + str(std_rain) + " " + str(mean_max_temp) + " " + str(std_max_temp) + " " + str(mean_min_temp) + " " + str(std_min_temp))
        # Check for NaNs in calculated values
        if np.isnan(mean_yield) or np.isnan(std_yield):
            print(f"Warning: NaN found in 'yield' statistics for class {index}.")
            continue
        if np.isnan(mean_rad) or np.isnan(std_rad):
            print(f"Warning: NaN found in 'radiation' statistics for class {index}.")
            continue
        if np.isnan(mean_rain) or np.isnan(std_rain):
            print(f"Warning: NaN found in 'rain' statistics for class {index}.")
            continue
        if np.isnan(mean_max_temp) or np.isnan(std_max_temp):
            print(f"Warning: NaN found in 'avg_max_temp' statistics for class {index}.")
            continue
        if np.isnan(mean_min_temp) or np.isnan(std_min_temp):
            print(f"Warning: NaN found in 'avg_min_temp' statistics for class {index}.")
            continue
        
        new_yields = []
        new_rads = []
        new_rains = []
        new_max_temps = []
        new_min_temps = []
        
        # calculate potcii: percentage of this class in input
        potcii = (len(entry)/no_records)
        no_records_to_generate = round(potcii * samples_out)
        print("Number of records to generate: ", no_records_to_generate)
        for i in range(no_records_to_generate):
            new_yield = random.uniform(mean_yield - std_yield*no_stds, mean_yield + std_yield*no_stds)
            new_yields.append(new_yield)
            #print("New Yields:")
            #print(new_yields)
            
            new_rad = random.uniform(mean_rad - std_rad*no_stds, mean_rad + std_rad*no_stds)
            new_rads.append(new_rad)
            #print("Radiation:")
            #print(new_rads)
            
            new_rain = random.uniform(mean_rain - std_rain*no_stds, mean_rain + std_rain*no_stds)
            new_rains.append(new_rain)
            #print("Precipitation:")
            #print(new_rains)
        
            new_max_temp = random.uniform(mean_max_temp - std_max_temp*no_stds, mean_max_temp + std_max_temp*no_stds)
            new_max_temps.append(new_max_temp)
            #print("Max Temp:")
            #print(new_max_temps)
            
            new_min_temp = random.uniform(mean_min_temp - std_min_temp*no_stds, mean_min_temp + std_min_temp*no_stds)
            new_min_temps.append(new_min_temp)
            #print("Min Temperature")
            #print(new_min_temps)
            
        # Create a DataFrame for the new synthesized data
        new_data = pd.DataFrame({
            'yield': new_yields,
            'radiation': new_rads,
            'rain': new_rains,
            'avg_max_temp': new_max_temps,
            'avg_min_temp': new_min_temps,
            'class': [index] * len(new_yields)  # Adding the class label
        })
        
        # Concatenate the original data with the new synthesized data
        concat_data = pd.concat([entry, new_data], ignore_index=True)
        
        # Append the concatenated DataFrame to the list
        new_F.append(concat_data)
    
    # Finally, concatenate all class DataFrames into a single DataFrame
    final_data = pd.concat(new_F, ignore_index=True)
    
    return final_data
    #return pd.concat(new_F)

new_data = synthesize_tabular_data(F, samples_out, no_stds, number_of_classes, data_len)



Number of records to generate:  3846
Number of records to generate:  11154
Number of records to generate:  5000


In [45]:
print(len(new_data))

20052


In [46]:
new_data.to_csv("data_no_2022_synthesized_nonstationary.csv", index=False)

In [47]:
target_Df.reset_index(drop=True, inplace=True)
all_yearsDf.reset_index(drop=True, inplace=True)


In [48]:
all_yearsDf.to_csv("all_years.csv", index=False)
target_Df.to_csv("target_yearDf.csv", index=False)